In [1]:
import json     # JSON Encode-Decode library
import numpy as np     # Scientific computational library
from sklearn.feature_extraction.text import TfidfVectorizer     # WordVecotr with TF-IDF  
from sklearn.ensemble import RandomForestClassifier     # Classifier

In [2]:
# Reading chat data
with open("chatdata.json", 'r') as f:
    chat_data = json.load(f)
    f.close()

In [3]:
# Training Data
training_dict = {}

# Appeding chat data in Training dicotionary
for intent, question_list in chat_data.items():
    for question in question_list:
        training_dict[question] = intent

In [4]:
# Separating Features i.e questions and Labels i.e intents
feature = np.array(list(training_dict.keys()))
labels = np.array(list(training_dict.values()))
feature, labels

(array(['hi', 'hello', 'hey', 'hola', 'bye', 'goodbye', 'good bye', 'sad',
        'not good', 'i am sad', 'not happy', 'i am happy', 'i am good',
        'great', 'good', 'how are you?', 'whats up', 'wassup',
        'what is your name', 'Hey who are you', 'tell me your name',
        'may i know your name'], dtype='<U20'),
 array(['greet', 'greet', 'greet', 'greet', 'goodbye', 'goodbye',
        'goodbye', 'sad', 'sad', 'sad', 'sad', 'hppay', 'hppay', 'hppay',
        'hppay', 'askbot', 'askbot', 'askbot', 'botname', 'botname',
        'botname', 'botname'], dtype='<U7'))

In [5]:
# Converting text to WordVector
tf_vec = TfidfVectorizer().fit(feature)
X = tf_vec.transform(feature).toarray()

# Reshaping labels to fit data
y = labels.reshape(-1,1)

In [6]:
# Fitting model
rnn = RandomForestClassifier(n_estimators=200)
rnn.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
# Reading Resposne 
with open("response.json") as f:
    response_dict = json.load(f)
    f.close()

# Creating response
def botanswer(q):
    process_text = tf_vec.transform([q]).toarray()
    prob = rnn.predict_proba(process_text)[0]
    max_ = np.argmax(prob)
    if prob[max_] < 0.6:
        return "Sorry I am not getting you...!"
    else:
        return response_dict[rnn.classes_[max_]]

In [9]:
# Chat with bot
while True:
    user = input("User>> ")
    if user == "quit":
        break
    print("Bot>> {}".format(botanswer(user)))

Bot>> Hello there...!
Bot>> I am fine, How are you?
Bot>> That's great
Bot>> Bye see you later
